# Project Title
### Data Engineering Capstone Project

#### Project Summary
The goal of this project is to generate a Datawarehouse for performing reports about immigration to the US. The special aspect of this datawarehouse is that it enables the analysis of immigrants within the US by, for example, analyzing the countries they come from. It will also be possible to study the distribution of immigrants within the US by finding out relevant statistcs of foreign borned american and the distribution of races in american cities. Furthermore, this datawarehouse will not only contain demographic statistics, but also environemtal and geographical information for broadening the scope of possible analyses which could be performed on this database.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [4]:
# Do all imports and installs here
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date, LongType as Long
from datetime import datetime
import datetime
import os
import numpy as np
import pandas as pd
import s3fs
import boto3
from io import StringIO
import configparser
%matplotlib inline
import matplotlib.pyplot as plt

In [5]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
KEY=config.get('AWS','key')
SECRET= config.get('AWS','secret')
os.environ['AWS_ACCESS_KEY_ID']=config.get('AWS','key')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('AWS','secret')

In [16]:
#spark1 = SparkSession.builder.\
#config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
#.enableHiveSupport().getOrCreate()
#df_spark =spark1.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

### Step 1: Scope the Project and Gather Data

#### Scope 
This projects examines the immigration patterns in the US. For this, 4 datasets are used that contain information not only of single persons entering the country, but also of the places they arrive at and the characteristics of these ports of entry. The data used is contained in an attached disk in Udacity. This data will be loaded by using pandas in order to analyze it and clean it to guarantee a good quality of it. Afterwards, these data will be saved to S3 buckets. The data contained in the S3 buckets will be copied into Redshift as staging tables, where they will be further processed until a star schema is obtained. Airflow will be used starting with the loading step to Redshift until the data quality checks are performed after the structuring of the star schema has succeded.

#### Datasets used in the project
Following datasets were used for this project:
1. I94 Immigration Data: This data lists all applications made to enter to the USA as an immigrant during 2016.
2. World Temperature Data: This dataset contains relevant information about climate at several cities around the world. The dataset contains the average temperature of cities and countries monthly calculated for a long period of time. The data comes from Kaggle
3. U.S. City Demographic Data: This dataset provides mainly demographic statistics of cities in the US. The information contained in this dataset include city, state, median age, population, race, among others. The data comes from OpenSoft
4. Airport Code Table: As the name says, this dataset provides all information related to airports around the world. The information contained here includes: type of airport, latitude, longitude, airport's city, among others. The data comes from datahun.io

### I94 Immigration: Data Loading

In [3]:
# Data in Udacity's attached disk
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

# Read in SAMPLE immigration data
#path = 'immigration_data_sample.csv'
#df = spark.read.format('csv').options(header='true').schema(immiSchema).load(path)
#df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(path)
#df = pd.read_csv(path)
#df.head(5)

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


### U.S. Cities Demographics: Data Loading

In [6]:
#Read in us cities demographics data
path_demo = 'us-cities-demographics.csv'
#df_demo = spark.read.format("csv").option("header","true").option("inferSchema","true").option("delimiter", ";").load(path_demo)
df_demo = pd.read_csv(path_demo, sep=';')
df_demo.head(5)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


### Temperatures by City: Data Loading

Now, we'd like to get a better insight into the GlobalLandTemperaturesByCity database. As follows, the data (in .csv form) will be loaded onto Spark

In [5]:
path_temp = '../../data2/GlobalLandTemperaturesByCity.csv'
#df_temp = spark.read.format("csv").option("header","true").option("inferSchema","true").load(path_temp)
df_temp = pd.read_csv(path_temp)
df_temp.head(5)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


### Airport Codes: Data Loading

In [3]:
path_air = 'airport-codes_csv.csv'
df_air = pd.read_csv(path_air)
df_air.head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


### Step 2: Explore and Assess the Data
#### Explore the Data 
The uploaded datasets uploaded in the previous step will be inspected for data quality issues. For this, it will be looked for missing values, duplicated values, irrelevant columns, etc.

#### Cleaning Steps
First of all, the size of the data is printed. After that, the schema of the datasets is printed in order to know the types of the data. Finally, it is analyzed which columns have to be not null and if duplicated values are allowed or not.

### I94 Immigration: Data Processing

It is assumed that the column "admnum" means "admission number" referring to an unique number assignated to every application submitted to the I94 system. For this reason, there could not be any null values nor duplicated ones. Similarly, column "cicid" is considered to be an unique identification number generated by the I94 system for every person applying for immigration. For this reason, this column cannot have NULL values, however it could have duplicated values since 1 person could have submitted multiples applications with different visa types

In [4]:
# Column "Unnamed" is deleted as well as not null values where they are not wanted
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df[df.admnum.notnull()]
df = df[df.cicid.notnull()]
df = df.drop_duplicates(subset=['admnum'])
df.head(5)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [25]:
#Conversion of string to datetime types in pandas
df['dtadfile'] = pd.to_datetime(df['dtadfile'], format='%Y%m%d')
df['dtaddto'] = pd.to_datetime(df['dtadfile'], format='%m%d%Y')

In [26]:
# Count of rows
df.shape[0]

1000

In [27]:
# Schema of data
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 28 columns):
cicid       1000 non-null float64
i94yr       1000 non-null float64
i94mon      1000 non-null float64
i94cit      1000 non-null float64
i94res      1000 non-null float64
i94port     1000 non-null object
arrdate     1000 non-null float64
i94mode     1000 non-null float64
i94addr     941 non-null object
depdate     951 non-null float64
i94bir      1000 non-null float64
i94visa     1000 non-null float64
count       1000 non-null float64
dtadfile    1000 non-null datetime64[ns]
visapost    382 non-null object
occup       4 non-null object
entdepa     1000 non-null object
entdepd     954 non-null object
entdepu     0 non-null float64
matflag     954 non-null object
biryear     1000 non-null float64
dtaddto     1000 non-null datetime64[ns]
gender      859 non-null object
insnum      35 non-null float64
airline     967 non-null object
admnum      1000 non-null float64
fltno       992 non-

### US cities demographics: Data Processing

As we can see, this data describes the demographics of every single city in the US. Important here is to verify that the tuple (city, state) are not duplicated, since it can be the case that 2 cities are called the same but they cannot coexist in the same state. Other than that, it will be verified that there are no null values in the column "city" or "state" since they will be the primary keys of this dataset

We'd like to know the size (number of entries) of the file just uploaded to the dataframe "df_demo"

In [16]:
df_demo.shape[0]

2891

In [17]:
df_demo.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


In [7]:
# Null values for columns "city" and "state" are deleted as well as duplicated values
df_demo = df_demo[df_demo.City.notnull()]
df_demo = df_demo[df_demo.State.notnull()]
df_demo = df_demo.drop_duplicates(subset=['City','State'])
df_demo.head(5)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [19]:
df_demo.shape[0]

596

### Temperatures by City: Data Processing

This table will complement the information contained in US Cities demographics by adding columns such as latitude, longitude and average temperature

In [20]:
df_temp.shape[0]

8599212

In [21]:
df_temp.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


In [22]:
df_temp = df_temp[df_temp.City.notnull()]
de_temp = df_temp[df_temp.Latitude.notnull()]
df_temp = df_temp[df_temp.Longitude.notnull()]
df_temp = df_temp.drop_duplicates(subset=['City','Country'])
df_temp.head(5)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
3239,1743-11-01,10.013,2.291,Çorlu,Turkey,40.99N,27.69E
6478,1753-01-01,-1.548,4.951,Çorum,Turkey,40.99N,34.08E
9607,1820-08-01,14.284,2.544,Öskemen,Kazakhstan,50.63N,82.39E
11925,1820-08-01,20.146,2.286,Ürümqi,China,44.20N,87.20E


In [23]:
df_temp.shape[0]

3490

### Airport Codes: Data Processing

This table is fundamental since it will be used to map the city abbreviation which is contained in i94 immigration table with the name of the corresponding city

In [4]:
df_air.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


In [5]:
df_air = df_air.drop(['coordinates','name'], axis=1)

In [6]:
df_air.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 10 columns):
ident           55075 non-null object
type            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
dtypes: float64(1), object(9)
memory usage: 4.2+ MB


In [7]:
df_air.size

550750

In [8]:
df_air.infer_objects().dtypes

ident            object
type             object
elevation_ft    float64
continent        object
iso_country      object
iso_region       object
municipality     object
gps_code         object
iata_code        object
local_code       object
dtype: object

In [9]:
# Since every airport has an identifier and they are all registered in a city, these fields cannot contain null values
df_air = df_air[df_air.municipality.notnull()]
df_air = df_air[df_air.iata_code.notnull()]
df_air.size

84230

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
The chosen data model for this project is a Data Warehouse. For this, a star schema was designed with 1 fact table and 3 dimension tables. All tables on the database fulfilled the 3NF conditions. The fact table is called "immigration" table and it contains all the records of aliens entering the USA during 2016. This table contains following columns: migration_id (PRIMARY KEY), persons_id, city_id, date_id, visa_type, mode_of_entry, visa_post, arrival_flag, departure_flag and update_flag. The 3 dimension tables are "persons", "dates" and "city". Columns for persons are: persons_id (PRIMARY KEY), birthyear, age, gender, occupation, country_of_origin, country_of_residence. Columns for dates are: date_id (PRIMARY KEY), day, week, month, year, weekday. Columns for city are: city_id (PRIMARY KEY), name, state, latitude, longitude, average_temperature, total_population, foreign_born.

#### 3.2 Mapping Out Data Pipelines
In order to get from the raw data provided to relational databases on Redshift, following steps have to be carried out:
1. After having cleaned and processed the data in the previous step (Explore and Asses the data), the resulting pandas dataframes are sent to corresponding S3 buckets in csv form
2. Data contained in the S3 bucket is then loaded/copied to staging tables in Redshift
3. Fact and dimension tables are filled with data joined from the staging tables
4. Finally, data quality checks are performed on the data in order to guarantee a high quality of the resulting relational tables

At this point, it is important to take into account that the process just described aboved is carried out by Airflow as it is shown in detail by the following illustration:

<img src="DAG_pic.PNG">

As described above, the first step in the pipeline is to send all the data from pandas dataframes to csv files in S3. This is achieved by running the cell below:

In [24]:
# To start with, the data loaded and clenaed in the previous steps will be loaded into a S3 Bucket
import boto3

s3_resource = boto3.resource('s3',
                       region_name="eu-central-1",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                     )

bucket = 'capstone-usimmigration'


csv_buffer = StringIO()
df.to_csv(csv_buffer)
s3_resource.Object(bucket, 'immigration.csv').put(Body=csv_buffer.getvalue())

csv_buffer_demo = StringIO()
df_demo.to_csv(csv_buffer_demo)
s3_resource.Object(bucket, 'us_demographics.csv').put(Body=csv_buffer_demo.getvalue())

csv_buffer_temp = StringIO()
df_temp.to_csv(csv_buffer_temp)
s3_resource.Object(bucket, 'temperaturesCity.csv').put(Body=csv_buffer_temp.getvalue())

csv_buffer_air = StringIO()
df_air.to_csv(csv_buffer_air, sep=";")
s3_resource.Object(bucket, 'airport_codes.csv').put(Body=csv_buffer_air.getvalue())

{'ResponseMetadata': {'RequestId': '31ECA082C145AD97',
  'HostId': 'Dj+zXXotxn8NizKfnV/mQQ5MSSum8ixOTDyj5SWiT8yhD17UsaGNVq5PHDCs84O7+qrVZUNzukM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Dj+zXXotxn8NizKfnV/mQQ5MSSum8ixOTDyj5SWiT8yhD17UsaGNVq5PHDCs84O7+qrVZUNzukM=',
   'x-amz-request-id': '31ECA082C145AD97',
   'date': 'Tue, 13 Oct 2020 12:06:33 GMT',
   'etag': '"389117bd83d6396560aac31c7ac0bc87"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"389117bd83d6396560aac31c7ac0bc87"'}

### Step 4: Run Pipelines to Model the Data 
Since the ETL pipeline for this project was developed on Airflow, refer to the code attached to this workspace under folder "airflow"
#### 4.1 Create the data model
By having succesfully loaded the csv files to S3, now these file are copied into staging Tables in Redshift. As can be seen in the illustration shown above, 4 staging tables are created namely: staging_immigration, staging_usdemographics, staging_climate and staging_airportcodes. For creating the tables in Redshift, the PostgresOperator in Airflow was used. On the other hand, for copying the data from S3 to Redshift, StageToRedshiftOperator was programmed.
Next, the tables of the star schema were created by using the Postgres Operator. In order to load the data from staging tables into fact and dimesion tables, operators LoadFactOperator and LoadDimesionOperator were used.
Finally, several quality checks are performed on every table aiming at getting a high quality of the data in the Warehouse

#### 4.2 Data Quality Checks
To ensure a succesfull run of the pipeline designed, in total 6 data quality checks were designed in this project for the 4 resulting fact and dimension tables. These checks were specially designed for ensuring completeness. Integrity constraints on the relational database are automatically performed by loading the right data into the correct format in Redshift. Finally, unit tests for the scripts are not necessary since this was performed in step 2 of the project. The 6 data quality checks are:
 * Checking that the number of states in the US contained in the "city" table are less or equal to 50
 * Ensuring that the number of entries in the "dates" table does not exceed 365 since we have data for only one year (2016)
 * Checking that the column "year" in "dates" tables only contains the integer 2016
 * Verifying that the age of applicants for residence in the US is under 120 years
 * Checking that there are no null values for column "migration_id" which corresponds to the primary key of table "immigration"
 * Checking that there are no null values for column "persons_id" which corresponds to the primary key of table "persons"

These quality checks are the last task executed by the pipeline built on Airflow

#### 4.3 Data dictionary 
As follows the data in every table of the star schema will be described:

##### 4.3.1 Fact Table: Immigration
* migration_id: admnum or application number. It comes from staging immigration
* persons_id: cicid or unique number for every person in the i94 dataset. It comes from staging immigration
* city_id: i94port or abbreviation of the US city the application was submitted for. It comes from staging immigration
* date_id: dtadfile or date when the application for residence was submitted onto the system. It comes from staging immigration
* visa_type: visatype. It comes from staging immigration
* mode_of_entry: i94mode or way of entry to the US. It comes from staging immigration
* visa_post: visapost or direction where the visa is to be sent to. It comes from staging immigration
* arrival_flag: entdepa and tells if the person was admitted to the US. It comes from staging immigration
* departure_flag: entdepd and tells if the person departed from the US. It comes from staging immigration
* update_flag: entdepu and tells if there was any update on the state of the application. It comes from staging immigration

##### 4.3.2 Dimension Table: City
* city_id: i94port or port of entry for application. It comes from staging immigration
* name: municipality or name of city. It comes from staging aiportcodes
* state: state of the city. It comes from staging usdemographics
* latitude: latitude of the city. It comes from staging climate
* longitude: longitude of the city. It comes from staging climate
* average_temperature: average_temprature of the city. It comes from staging climate
* total_population: total_population of the city. It comes from staging usdemographics
* foreign_born: foreign_born and tells the number of persons borned from foreigners. It comes from staging usdemographics

##### 4.3.3 Dimension Table: Persons
* persons_id: cicid or personal identification number. It comes from staging immigration
* birthyear: i94yr is the birthyear of the applicant. It comes from staging immigration
* age: i94bir is the age of the applicant. It comes from staging immigration
* gender: gender of the applicant. It comes from staging immigration
* occupation: occup which means occupation of the applicant. It comes from staging immigration
* country_of_origin: i94cit or code of the country of origin of the applicant. It comes from staging immigration
* country_of_residence: i94res or code for the country of residence of the applicant. It comes from staging immigration

##### 4.3.4 Dimension Table: Dates
* date_id: dtadfile denotes the dates when the application was submitted to the i94 immigration system. It comes from staging immigration
* day: day of application's submission date. It comes from staging immigration
* week: week of application's submission date. It comes from staging immigration
* month: month of application's submission date. It comes from staging immigration
* year: year of application's submission date. It comes from staging immigration
* weekday: weekday of application's submission date. It comes from staging immigration

#### Step 5: Complete Project Write Up
* Since a datawarehouse was evaluated to be the best solution for this project, Redshift was used as it rovides tools for managing this kind of data. Other than that, given that we were dealing with relational tables and we have to execute several data migration steps, Airflow as used to model the pipeline in order to have an overview of the state of the many steps involved in the reation of the datawarehouse. Finally, pandas is used for inspecting and cleaning the data in the first steps as it provides appropiate tools for handling data frames. S3 buckets were used for storing the data as they are standard for data management purposes nowadays. Also, in case the amount of the data increases in an exponential way, this repository can grow accordingly.
* The data should be updated in a monthly basis, since decisions for residence permit take a while to process by the immigration office.
* Write a description of how you would approach the problem differently under the following scenarios:
 * If the data was increased by 100x, I would partition the data coming from i94 by year and month since it is the only way to distribute the data in approximately equal size.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day: Since dashboards contain specific KPIs which are the result of optimized queries, I would build in this case a NoSQL database where each table is optimized for a specific query
 * The database needed to be accessed by 100+ people: All users could access the data warehouse with the current configuration of the project